In [34]:
# import tensorflow.compat.v1 as tf      # Deep Learning library
import numpy as np           # Handle matrices
from vizdoom import *        # Doom Environment
# from tensorflow.python.framework import ops
import vizdoom as vzd
import numpy as np
import torch
import json
import torch.nn as nn
import torch.nn.functional as F
from torch import nn, optim
import cv2

import random                 # Handling random number generation
import time                   # Handling time calculation
from skimage import transform # Help us to preprocess the frames

from collections import deque # Ordered collection with ends
import matplotlib.pyplot as plt  # Display graphs

import warnings                  # This ignore all the warning messages that are normally printed during the training because of skiimage
warnings.filterwarnings('ignore')

#tf.disable_eager_execution()

In [35]:
### MODEL HYPERPARAMETERS
state_size = [240,320,4]      # Our input is a stack of 4 frames hence 100x120x4 (Width, height, channels)
action_size = 3          # 3 possible actions
learning_rate =  0.00025      # Alpha (aka learning rate)

### TRAINING HYPERPARAMETERS
total_episodes = 1000000         # Total episodes for training
# max_steps = 100000            # Max possible steps in an episode
batch_size = 128
save_interval = 50
# FIXED Q TARGETS HYPERPARAMETERS
max_tau = 1000 #Tau is the C step where we update our target network
clip_norm = 0.001

# EXPLORATION HYPERPARAMETERS for epsilon greedy strategy
explore_start = 1.0            # exploration probability at start
explore_stop = 0.001            # minimum exploration probability
decay_rate = 0.000005          # exponential decay rate for exploration prob

# Q LEARNING hyperparameters
gamma = 0.99               # Discounting rate

### MEMORY HYPERPARAMETERS
## If you have GPU change to 1million
pretrain_length = 5000             # Number of experiences stored in the Memory when initialized for the first time
memory_size = 100000 ##100000                 # Number of experiences the Memory can keep


In [36]:


def create_environment():
    game = vzd.DoomGame()

    # Load the correct configuration
    # game.load_config('./VizDoom/scenarios/defend_the_center.cfg')

    # Load the correct scenario (in our case defend_the_center)
    # game.set_screen_format(vzd.ScreenFormat.RGB24)
    # game.set_screen_resolution(vzd.ScreenResolution.RES_160X120)
    game.set_depth_buffer_enabled(True) # depth buffer

    game.set_labels_buffer_enabled(True) # labeling of game objects in labeling

    game.set_automap_buffer_enabled(True) # Enables buffer with top down map of the current episode/level.

    game.set_objects_info_enabled(True) # Information of all obects present in the current episode level 

    game.set_sectors_info_enabled(True) # Enables information about all sectors (map layout).

    # Disable rendering options
    game.set_render_hud(False)
    game.set_render_minimal_hud(False)  
    game.set_render_crosshair(False)
    game.set_render_weapon(True)
    game.set_render_decals(False)  
    game.set_render_particles(False)
    game.set_render_effects_sprites(False) 
    game.set_render_messages(False)  
    game.set_render_corpses(False)
    game.set_render_screen_flashes(True) 

    game.set_episode_start_time(1)
    game.set_mode(vzd.Mode.PLAYER)
    game.set_doom_scenario_path('./VizDoom/scenarios/defend_the_center.wad')
    game.set_doom_map("map01")
    game.set_available_game_variables([vzd.GameVariable.HEALTH, vzd.GameVariable.AMMO2])
    game.set_available_buttons([vzd.Button.TURN_LEFT, vzd.Button.TURN_RIGHT, vzd.Button.ATTACK])

    game.set_window_visible(False) #no pop out window
    # game.init()

    # Here we create an hot encoded version of our actions (3 possible actions)
    # possible_actions = [[1, 0, 0, 0, 0], [0, 1, 0, 0, 0]...]
    possible_actions = np.identity(3,dtype=int).tolist()

    return game, possible_actions






def preprocess_frame(frame):
    # Crop the screen (remove part that contains no information)
    # [Up: Down, Left: right]
    # # Check if the cropped frame has non-zero dimensions
    # if cropped_frame.size == 0:
    #     print("A")
    #     # If the cropped frame has zero dimensions, return a default frame with zeros
    #     return np.zeros((100, 120), dtype=np.float32)
    
    # print("b")
    # Normalize Pixel Values
    if frame.shape == (100, 120):
        return frame
    # print("frame to preprocess: ", frame.shape)
    gray = cv2.cvtColor(np.moveaxis(frame, 0, -1), cv2.COLOR_BGR2GRAY)
    resize = cv2.resize(gray, (120,100), interpolation=cv2.INTER_CUBIC)
    
    normalized_frame = resize/255.0
    #print("processed shape: ", normalized_frame.shape)

    # plt.imshow(resize, cmap='gray')  # cmap='gray' for grayscale
    # plt.axis('off')  # Turn off axis
    # plt.show()
    
    return normalized_frame # 100x120x1 frame






def stack_frames(stacked_frames, state, is_new_episode):
    # Preprocess frame
    frame = preprocess_frame(state)
    
    if is_new_episode:
        # Clear our stacked_frames
        stacked_frames = deque([np.zeros((100,120), dtype=np.float32) for i in range(stack_size)], maxlen=4)
        
        # Because we're in a new episode, copy the same frame 4x
        stacked_frames.append(frame)
        stacked_frames.append(frame)
        stacked_frames.append(frame)
        stacked_frames.append(frame)
        
        # Stack the frames
        stacked_state = np.stack(stacked_frames, axis=2)

    else:
        # Append frame to deque, automatically removes the oldest frame
        stacked_frames.append(frame)

        # Build the stacked state (first dimension specifies different frames)
        stacked_state = np.stack(stacked_frames, axis=2) 
    
    return stacked_state, stacked_frames




game, possible_actions = create_environment()

stack_size = 4 

# Initialize deque with zero-images one array for each image
stacked_frames  =  deque([np.zeros((100,120), dtype=np.float32) for i in range(stack_size)], maxlen=4) 

In [37]:
class DDDQNNet(nn.Module):
    def __init__(self, image_height, image_width, num_actions):
        super().__init__()
        h = image_height
        w = image_width
        self.c1 = nn.Conv2d(in_channels=4, out_channels=16, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=4)
        h = h // 4
        w = w // 4
        self.c2 = nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=4)
        h = h // 4
        w = w // 4
        
        self.fc_value = nn.Linear(h * w * 16, 512)
        self.value = nn.Linear(512, 1)
        
        self.fc_advantage = nn.Linear(h * w * 16, 512)
        self.advantage = nn.Linear(512, num_actions)
        
    def forward(self, x):
        x = self.c1(x)
        x = self.pool1(x)
        x = F.relu(x)

        x = self.c2(x)
        x = self.pool2(x)
        x = F.relu(x)
        
        x = x.view(x.size(0), -1)
        
        value = F.relu(self.fc_value(x))
        value = self.value(value)
        
        advantage = F.relu(self.fc_advantage(x))
        advantage = self.advantage(advantage)
        
        # Combining value and advantage streams to get output (Q values)
        output = value + (advantage - advantage.mean(dim=1, keepdim=True))
        #print("network output shape: ", output.shape)  # tensor[1,3]
        return output


In [38]:
class SumTree(object):
    """
    This SumTree code is modified version of Morvan Zhou: 
    https://github.com/MorvanZhou/Reinforcement-learning-with-tensorflow/blob/master/contents/5.2_Prioritized_Replay_DQN/RL_brain.py
    """
    data_pointer = 0
    
    """
    Here we initialize the tree with all nodes = 0, and initialize the data with all values = 0
    """
    def __init__(self, capacity):
        self.capacity = capacity # Number of leaf nodes (final nodes) that contains experiences
        
        # Generate the tree with all nodes values = 0
        # To understand this calculation (2 * capacity - 1) look at the schema above
        # Remember we are in a binary node (each node has max 2 children) so 2x size of leaf (capacity) - 1 (root node)
        # Parent nodes = capacity - 1
        # Leaf nodes = capacity
        self.tree = np.zeros(2 * capacity - 1)
        
        """ tree:
            0
           / \
          0   0
         / \ / \
        0  0 0  0  [Size: capacity] it's at this line that there is the priorities score (aka pi)
        """
        
        # Contains the experiences (so the size of data is capacity)
        self.data = np.zeros(capacity, dtype=object)
    
    
    """
    Here we add our priority score in the sumtree leaf and add the experience in data
    """
    def add(self, priority, data):
        # Look at what index we want to put the experience
        tree_index = self.data_pointer + self.capacity - 1
        
        """ tree:
            0
           / \
          0   0
         / \ / \
tree_index  0 0  0  We fill the leaves from left to right
        """
        
        # Update data frame
        self.data[self.data_pointer] = data
        
        # Update the leaf
        self.update (tree_index, priority)
        
        # Add 1 to data_pointer
        self.data_pointer += 1
        
        if self.data_pointer >= self.capacity:  # If we're above the capacity, you go back to first index (we overwrite)
            self.data_pointer = 0
            
    
    """
    Update the leaf priority score and propagate the change through tree
    """
    def update(self, tree_index, priority):
        # Change = new priority score - former priority score
        change = priority - self.tree[tree_index]
        self.tree[tree_index] = priority
        
        # then propagate the change through tree
        while tree_index != 0:    # this method is faster than the recursive loop in the reference code
            
            """
            Here we want to access the line above
            THE NUMBERS IN THIS TREE ARE THE INDEXES NOT THE PRIORITY VALUES
            
                0
               / \
              1   2
             / \ / \
            3  4 5  [6] 
            
            If we are in leaf at index 6, we updated the priority score
            We need then to update index 2 node
            So tree_index = (tree_index - 1) // 2
            tree_index = (6-1)//2
            tree_index = 2 (because // round the result)
            """
            tree_index = (tree_index - 1) // 2
            self.tree[tree_index] += change
    
    
    """
    Here we get the leaf_index, priority value of that leaf and experience associated with that index
    """
    def get_leaf(self, v):
        """
        Tree structure and array storage:
        Tree index:
             0         -> storing priority sum
            / \
          1     2
         / \   / \
        3   4 5   6    -> storing priority for experiences
        Array type for storing:
        [0,1,2,3,4,5,6]
        """
        parent_index = 0
        
        while True: # the while loop is faster than the method in the reference code
            left_child_index = 2 * parent_index + 1
            right_child_index = left_child_index + 1
            
            # If we reach bottom, end the search
            if left_child_index >= len(self.tree):
                leaf_index = parent_index
                break
            
            else: # downward search, always search for a higher priority node
                
                if v <= self.tree[left_child_index]:
                    parent_index = left_child_index
                    
                else:
                    v -= self.tree[left_child_index]
                    parent_index = right_child_index
            
        data_index = leaf_index - self.capacity + 1

        return leaf_index, self.tree[leaf_index], self.data[data_index]
    
    @property
    def total_priority(self):
        return self.tree[0] # Returns the root node
    


    
class Memory(object):  # stored as ( s, a, r, s_ ) in SumTree
    """
    This SumTree code is modified version and the original code is from:
    https://github.com/jaara/AI-blog/blob/master/Seaquest-DDQN-PER.py
    """
    PER_e = 0.01  # Hyperparameter that we use to avoid some experiences to have 0 probability of being taken
    PER_a = 0.6  # Hyperparameter that we use to make a tradeoff between taking only exp with high priority and sampling randomly
    PER_b = 0.4  # importance-sampling, from initial value increasing to 1
    
    PER_b_increment_per_sampling = 0.001
    
    absolute_error_upper = 1.  # clipped abs error

    def __init__(self, capacity):
        # Making the tree 
        """
        Remember that our tree is composed of a sum tree that contains the priority scores at his leaf
        And also a data array
        We don't use deque because it means that at each timestep our experiences change index by one.
        We prefer to use a simple array and to overwrite when the memory is full.
        """
        self.tree = SumTree(capacity)
        
    """
    Store a new experience in our tree
    Each new experience have a score of max_prority (it will be then improved when we use this exp to train our DDQN)
    """
    def store(self, experience):
        # Find the max priority
        max_priority = np.max(self.tree.tree[-self.tree.capacity:])
        
        # If the max priority = 0 we can't put priority = 0 since this exp will never have a chance to be selected
        # So we use a minimum priority
        if max_priority == 0:
            max_priority = self.absolute_error_upper
        
        self.tree.add(max_priority, experience)   # set the max p for new p

        
    """
    - First, to sample a minibatch of k size, the range [0, priority_total] is / into k ranges.
    - Then a value is uniformly sampled from each range
    - We search in the sumtree, the experience where priority score correspond to sample values are retrieved from.
    - Then, we calculate IS weights for each minibatch element
    """
    def sample(self, n):
        # Create a sample array that will contains the minibatch
        memory_b = []
        
        b_idx, b_ISWeights = np.empty((n,), dtype=np.int32), np.empty((n, 1), dtype=np.float32)
        
        # Calculate the priority segment
        # Here, as explained in the paper, we divide the Range[0, ptotal] into n ranges
        priority_segment = self.tree.total_priority / n       # priority segment
    
        # Here we increasing the PER_b each time we sample a new minibatch
        self.PER_b = np.min([1., self.PER_b + self.PER_b_increment_per_sampling])  # max = 1
        
        # Calculating the max_weight
        p_min = np.min(self.tree.tree[-self.tree.capacity:]) / self.tree.total_priority
        max_weight = (p_min * n) ** (-self.PER_b)
        
        for i in range(n):
            """
            A value is uniformly sample from each range
            """
            a, b = priority_segment * i, priority_segment * (i + 1)
            value = np.random.uniform(a, b)
            
            """
            Experience that correspond to each value is retrieved
            """
            index, priority, data = self.tree.get_leaf(value)
            
            #P(j)
            sampling_probabilities = priority / self.tree.total_priority
            
            #  IS = (1/N * 1/P(i))**b /max wi == (N*P(i))**-b  /max wi
            b_ISWeights[i, 0] = np.power(n * sampling_probabilities, -self.PER_b)/ max_weight
                                   
            b_idx[i]= index
            
            experience = [data]
            
            memory_b.append(experience)
        
        return b_idx, memory_b, b_ISWeights
    
    """
    Update the priorities on the tree
    """
    def batch_update(self, tree_idx, abs_errors):
        abs_errors += self.PER_e  # convert to abs and avoid 0
        clipped_errors = np.minimum(abs_errors, self.absolute_error_upper)
        ps = np.power(clipped_errors, self.PER_a)

        for ti, p in zip(tree_idx, ps):
            self.tree.update(ti, p)

In [39]:
# Instantiate memory
memory = Memory(memory_size)
import numpy as np 
from skimage import transform

# Render the environment
game.init()
game.new_episode()

for i in range(pretrain_length):
    # If it's the first step
    if i == 0:
        # First we need a state
        state = game.get_state().screen_buffer
        #print("raw state shape, ", state.shape) # [3, 240, 320]
        #print("raw state shape to torch, ", torch.from_numpy(state).shape) # torch[[3, 240, 320]]
        state, stacked_frames = stack_frames(stacked_frames, state, True) ### All stacked frames are preprocessed
    
    # Random action
    action = random.choice(possible_actions)
    
    # Get the rewards
    reward = game.make_action(action)
    
    # Look if the episode is finished
    done = game.is_episode_finished()

    # If we're dead
    if done:
        # We finished the episode
        next_state = np.zeros(state.shape)
        
        # Add experience to memory
        #experience = np.hstack((state, [action, reward], next_state, done))
        
        experience = state, action, reward, next_state, done
        #print("shape of a stack frame of dead: ", state.shape)
        memory.store(experience)
        
        # Start a new episode
        game.new_episode()
        
        # First we need a state
        state = game.get_state().screen_buffer
        
        # Stack the frames
        state, stacked_frames = stack_frames(stacked_frames, state, True)
        
    else:
        # Get the next state
        next_state = game.get_state().screen_buffer
        next_state, stacked_frames = stack_frames(stacked_frames, next_state, False)
        
        # Add experience to memory
        experience = state, action, reward, next_state, done
        #print("shape of a stack frame still alive: ", state.shape)
        memory.store(experience)
        
        # Our state is now the next_state
        state = next_state
        test_state = state

game.close()


In [40]:
# Instantiate the DQNetwork
PolicyNetwork = DDDQNNet(100, 120, 3)

# Instantiate the target network
TargetNetwork = DDDQNNet(100, 120, 3)

optimizer = optim.RMSprop(PolicyNetwork.parameters(), lr=learning_rate)

# Initilise stack size
stack_size = 4
# Initialize deque with zero-images one array for each image
stacked_frames  =  deque([np.zeros((100,120), dtype=float) for i in range(stack_size)], maxlen=4)

In [41]:

def stacked_frame_to_torch_tensor(stacked_frame):
    '''
    input: [100, 120, 4]
    output: torch[[4, 100, 120]]
    '''
    torch_tensor = torch.from_numpy(stacked_frame).float()
    torch_tensor = torch_tensor.transpose(1, 2)
    torch_tensor = torch_tensor.transpose(0, 1)
    return torch_tensor.unsqueeze(0)


def select_action(explore_start, explore_stop, decay_rate, decay_step, state):
    """
    This function will do the part
    With ϵ select a random action atat, otherwise select at=argmaxaQ(st,a)

    Input state is 4 stacked states. 
    """

    num = np.random.rand()

    # Get explore probability
    explore_probability = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * decay_step)

    if (num < explore_probability):
        action = random.choice(possible_actions)
        return action, explore_probability

    # Greedy action
    else:
        max_index = PolicyNetwork(stacked_frame_to_torch_tensor(state)).argmax(dim=1).item()
        action = possible_actions[max_index]
        return action, explore_probability


def update_target_params(PolicyNetwork, TargetNetwork):
    """
    Copy parameters of the Policy network to Target network.
    """
    TargetNetwork_state_dict = TargetNetwork.state_dict()
    PolicyNetwork_state_dict = PolicyNetwork.state_dict()
    for key in PolicyNetwork_state_dict:
        TargetNetwork_state_dict[key] = PolicyNetwork_state_dict[key]
        TargetNetwork.load_state_dict(TargetNetwork_state_dict)

    return PolicyNetwork, TargetNetwork    

In [42]:
import vizdoom as vzd
# Set decay step
decay_step = 0
# Set tau 
tau = 0
# Set Target network params
PolicyNetwork, TargetNetwork = update_target_params(PolicyNetwork, TargetNetwork)

out_f = open("./log_PyTorch_DDDQN.txt", 'w')

# Initialise new game instance
game, possible_actions = create_environment()
game.init()

for episode in range(total_episodes):

    episode_step = 0
    episode_reward = 0
    game.new_episode()

    state = game.get_state().screen_buffer
    # print(type(state))
    state, stacked_frames = stack_frames(stacked_frames, state, True)
    # why stack_frames turned dimensions to 100X120
    
    while not game.is_episode_finished():

        episode_step += 1
        tau += 1
        decay_step +=1
        action, explore_probability = select_action(explore_start, explore_stop, decay_rate, decay_step, state) # the state input here is a stacked state already
        
        reward = game.make_action(action)
        done = game.is_episode_finished()
        episode_reward += reward

        # if episode finished
        if done: 
            next_state = np.zeros((100,120), dtype=float)
            next_state, stacked_frames = stack_frames(stacked_frames, next_state, False)

            # Add experience to memory
            experience = state, action, reward, next_state, done
            memory.store(experience)

        else:
            # Get the next state
            next_state = game.get_state().screen_buffer
            
            # Stack the frame of the next_state (Note the output next_state is stacked)
            next_state, stacked_frames = stack_frames(stacked_frames, next_state, False)
    

            # Add experience to memory
            experience = state, action, reward, next_state, done
            memory.store(experience)
            
            # st+1 is now our current state
            state = next_state 


        ### LEARNING      
        # Obtain random mini-batch from memory
        tree_idx, batch, ISWeights_mb = memory.sample(batch_size)
        
        batch_states = torch.cat([stacked_frame_to_torch_tensor(each[0][0]) for each in batch])

        batch_actions = torch.FloatTensor([each[0][1] for each in batch])
        batch_rewards = torch.FloatTensor([each[0][2] for each in batch]).unsqueeze(1)
        batch_next_states = torch.cat([stacked_frame_to_torch_tensor(each[0][3]) for each in batch] ) # stacked frames of np arrays 
        batch_dones = torch.FloatTensor([each[0][4] for each in batch]).unsqueeze(1)

        actions_index = batch_actions.detach().numpy().flatten()

        with torch.no_grad():
            policy_q_next = PolicyNetwork(batch_next_states)
            target_q_next = TargetNetwork(batch_next_states)
            online_max_action = torch.argmax(policy_q_next, dim=1, keepdim=True)
            y = batch_rewards + (1 - batch_dones) * gamma * target_q_next.gather(1, online_max_action.long())


        loss = F.mse_loss(PolicyNetwork(batch_states).gather(1, batch_actions.long()), y)
        optimizer.zero_grad()
        loss.backward()
        # torch.nn.utils.clip_grad_value_(PolicyNetwork.parameters(), clip)
        torch.nn.utils.clip_grad_norm_(PolicyNetwork.parameters(), clip_norm)
        optimizer.step()

        terminal_np = batch_dones.detach().numpy().flatten()
        policy_q_next_np = policy_q_next.detach().numpy()
        target_q_next_np = target_q_next.detach().numpy()
        rewards_np = batch_rewards.detach().numpy().flatten()

        target_qs_batch = []

        for i in range(0, len(batch)):

            terminal = terminal_np[i]
            action = np.argmax(policy_q_next_np[i])

            if terminal:
                target_qs_batch.append(torch.tensor(rewards_np[i], dtype=torch.float32)) # rewards_mb[i] is a TENSOR

            else: 
                target = rewards_np[i] + gamma * target_q_next_np[i][action]

                target_qs_batch.append(torch.tensor(target, dtype=torch.float32))

        ACTIONS = [torch.tensor(np.array([1,0,0]), dtype=torch.float32), torch.tensor(np.array([0,1,0]), dtype=torch.float32), torch.tensor(np.array([0,0, 1]), dtype=torch.float32)]

        predicted_qs = [torch.sum(torch.mul(policy_q_next[i], ACTIONS[int(actions_index[i])])) for i in range(batch_size)]

        absolute_errors = [torch.abs(torch.subtract(predicted_qs[i], target_qs_batch[i])) for i in range(batch_size)]
        
        absolute_errors_np = np.array([error.item() for error in absolute_errors])
        # print(absolute_errors_np)
        ISWeights_list = ISWeights_mb.tolist()
        memory.batch_update(tree_idx, absolute_errors_np)

        
        if tau > max_tau:
            PolicyNetwork, TargetNetwork = update_target_params(PolicyNetwork, TargetNetwork)
            tau = 0
            # print("Model updated")

    # Write batch stats to log files 
    loss_at_end_of_episode = torch.mean(torch.stack(absolute_errors), dim=0).item()
    print(f"Episode {episode}     |      Reward: {episode_reward}     |     length: {episode_step}      |      Loss: {loss_at_end_of_episode}")      
    
    # Write file to log
    out_f.write(json.dumps({
        'episode': episode,
        'reward': episode_reward,
        'length': episode_step,
        'loss': loss_at_end_of_episode
    }) + '\n')

    out_f.flush()
    
    if episode % save_interval == 0:
        torch.save(PolicyNetwork, f'./models/episode_{episode}.pt')
        print(f"======Model saved.======")
    # print(f"Episode: {episode}   |    Reward: {episode_reward}   |    Length: {episode_step}")


Episode 0     |      Reward: 0.0     |     length: 285      |      Loss: 0.07130824029445648
======Model saved.======
Episode 1     |      Reward: 0.0     |     length: 273      |      Loss: 0.04678485915064812
Episode 2     |      Reward: 0.0     |     length: 249      |      Loss: 0.06714797765016556
Episode 3     |      Reward: 1.0     |     length: 359      |      Loss: 0.040310099720954895
Episode 4     |      Reward: 1.0     |     length: 311      |      Loss: 0.06556517630815506
Episode 5     |      Reward: 2.0     |     length: 427      |      Loss: 0.07729307562112808
Episode 6     |      Reward: 3.0     |     length: 439      |      Loss: 0.07204821705818176
Episode 7     |      Reward: 0.0     |     length: 265      |      Loss: 0.07819285988807678
Episode 8     |      Reward: 1.0     |     length: 293      |      Loss: 0.07179142534732819


SignalException: Signal SIGINT received. ViZDoom instance has been closed.